# Description of analysis

I use the beach pollution data (1994-2021) to find the UK beaches with sufficient number of pre and post observations, considering 2015 as the year of policy change - the year when England began charging 5p for sigle use plastic bags. Here are some important dates to remember: 
* A five pence charge came into effect on single use carrier bags in England on 5 October 2015.

* Wales, Northern Ireland, and Scotland introduced a 5p levy on single use carrier bags in 2011, 2013, and 2014 respectively. The purpose of each single use carrier bag charge is to reduce the number of bags given out, increase their re-use and reduce litter.

Source: https://commonslibrary.parliament.uk/research-briefings/cbp-7241/


In [1]:
# Loading packages and data
import pandas as pd
import numpy as np

df = pd.read_excel("Beachwatch_AllData_1994-23.01.21inc Public Source Litter YChen.xlsx")

To include sufficent pre and post periods, I am only include beach pollution data between January 01, 2010 and December 31, 2018. We chose to exclude 2019 to avoid the pandemic years. 

In [2]:
start_date = pd.to_datetime('2010-01-01')
end_date = pd.to_datetime('2018-12-31')

df['Date of Survey'] = pd.to_datetime(df['Date of Survey'])

df = df[(df["Date of Survey"]>=start_date) & (df["Date of Survey"]<=end_date)]

df = df.reset_index(drop = True)

After filtering the dataset for the 2010 - 2018 period, the resulting data set has 7,250 observations for a total of 1,628 beaches.

In [3]:
# number of beahces
len(np.unique(df.BeachID))

1628

In [4]:
# number of observations
len(df)

7250

If we used monthly data for the above 1,628 beaches, we should have 175,824 observations [(9 x 12) x 1628]. But the current data appears to have only 7,250 observations. Perhaps, we should use yearly data instead of monthly.

### Converting to yearly data

The new dataset below, ndf, now contains beach pollution in yearly frequency. 

In [5]:
ndf  = df.sort_values(by = ['BeachID','Date of Survey'], ascending=[True, True]) #sorting
ndf= ndf.set_index('Date of Survey')
ndf = ndf.groupby('BeachID').resample('Y').sum() # converting to yearly average data
del ndf["BeachID"]
ndf = ndf.reset_index(level=1).reset_index() # resetting index

Now, considering October 2015 as the month of policy change, each beach should have maximum 5 years of pre-2015 observations and maximum 4 years of post-2015 (including 2015) observations. Below I subset the datset, including only the beaches that have at least 3 years' observatioins (total 6 years) from each of pre and post periods. 

In [6]:
# Separating the dataset for pre and post period

intervention = pd.to_datetime('2015-10-05') # the date when UK began the 5p charge
post_period = ndf[ndf['Date of Survey'] >= intervention]
pre_period = ndf[ndf['Date of Survey'] < intervention]

pre_observations_per_beach = pre_period.groupby('BeachID').size()
pre_observations_per_beach = pre_observations_per_beach.reset_index()
pre_observations_per_beach.columns = ['BeachID', '#of_Pre_observations']

post_observations_per_beach = post_period.groupby('BeachID').size()
post_observations_per_beach = post_observations_per_beach.reset_index()
post_observations_per_beach.columns = ['BeachID', '#of_Post_observations']

## Keeping only the beaches that have at least 3 years of pre data out of 5 years
pre_observations_per_beach = pre_observations_per_beach[pre_observations_per_beach['#of_Pre_observations']>=3]

## Keeping only the beaches that have at least 3 years of post data out of 4 years
post_observations_per_beach = post_observations_per_beach[post_observations_per_beach['#of_Post_observations']>=3]

The merged data set below shows the BeachID's with at least 3 years of pre and 3 years of post periods

In [7]:
## Merging the two subsets above
merged = pd.merge(pre_observations_per_beach, post_observations_per_beach, on = 'BeachID')

In [8]:
print(merged.to_string())

     BeachID  #of_Pre_observations  #of_Post_observations
0       2063                     4                      4
1       2065                     5                      4
2       2070                     5                      4
3       2084                     5                      4
4       2089                     5                      4
5       2095                     5                      3
6       2097                     4                      4
7       2101                     5                      4
8       2104                     5                      4
9       2107                     4                      4
10      2109                     5                      4
11      2115                     5                      4
12      2118                     4                      4
13      2119                     5                      4
14      2122                     5                      4
15      2123                     3                      4
16      2128  